<h1>logistic regression without applying polynomin</h1>
# this one should use less RAM than last version(v001)

In [30]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures


# df = pd.read_csv('./train.csv', index_col = True)
url = 'https://raw.githubusercontent.com/lucascheng24/comp4432ML-Dont-over-fit-II/main/train.csv'
df = pd.read_csv(url, index_col="id")
df['target'] = df['target'].astype(int)

parameters_dictionary = {}

for element in df:
    if element != "target":
        parameters_dictionary.update({element : df[element]})



label_dictionary = {'label': df['target']}

df_X = pd.DataFrame(data = parameters_dictionary)
df_Y = pd.DataFrame(data = label_dictionary)


# print(df_Y)

# print(df_Y.groupby('label').size())



In [31]:
# implement logistic regression


###
#   function: predict_logistic_regression
#       @param: input_x => training dataset features
#       @param: input_y => training dataset label
#       @param: input_z => testing dataset records (features only)
#   
#   output: array/list of predicted result of input_z
###
model = LogisticRegression(max_iter=10000)

def predict_logistic_regression(input_x, input_y, input_z):
    # build the model based on training data
    model.fit(input_x, input_y)

    # insert testing dataset to get the predicted output
    predicted_values = model.predict(input_z)

    return predicted_values
    


In [32]:
#  Applying 5-fold-cross-validation
k = 5
kf = KFold(n_splits = k, random_state = None)


# A function to run 5-fold cross validation on polynomial_logistic_regression
def kFold_logistic_regression(input_X, input_Y):

    validation_f1_score = []

    for train_index , validation_index in kf.split(input_X):

        # print('Start model fitting, k=', train_index[0], ' - ', train_index[-1], ' ...')

        X_train , X_valid = input_X.iloc[train_index,:], input_X.iloc[validation_index,:]
        y_train, y_valid = input_Y.iloc[train_index], input_Y.iloc[validation_index]

        # print('X_train: ', X_train)

        y_train = y_train.to_numpy().ravel()
        y_valid = y_valid.to_numpy().ravel()
        
        pred_testValues = predict_logistic_regression(X_train, y_train, X_valid)

        # count the f1 score(true records / predicted records)
        valid_f1 = f1_score(y_valid, pred_testValues)
        validation_f1_score.append(valid_f1)

    return validation_f1_score


In [35]:
testing_f1 = []

testing_f1_score = kFold_logistic_regression(df_X, df_Y)
testing_f1.append(testing_f1_score)


# print('testing set F1 score (poly 2 to 12) under 5-fold-cross-validation: ')
for row in testing_f1:
    print(row)

[0.8055555555555555, 0.8059701492537314, 0.7868852459016393, 0.71875, 0.7272727272727273]


<h3>Real prediction on testing data</h3>

In [38]:
import csv


url_test = 'https://raw.githubusercontent.com/lucascheng24/comp4432ML-Dont-over-fit-II/main/test.csv'
df_test = pd.read_csv(url_test, index_col="id")

print(df_test)

pred_values = predict_logistic_regression(df_X, df_Y, df_test)



with open('./result_folder/logisticV2.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["id", "target"])
    for i in range(250, 20000):
        writer.writerow([i, pred_values[i-250]])
    file.close()


           0      1      2      3      4      5      6      7      8      9  \
id                                                                            
250    0.500 -1.033 -1.595  0.309 -0.714  0.502  0.535 -0.129 -0.687  1.291   
251    0.776  0.914 -0.494  1.347 -0.867  0.480  0.578 -0.313  0.203  1.356   
252    1.750  0.509 -0.057  0.835 -0.476  1.428 -0.701 -2.009 -1.378  0.167   
253   -0.556 -1.855 -0.682  0.578  1.592  0.512 -1.419  0.722  0.511  0.567   
254    0.754 -0.245  1.173 -1.623  0.009  0.370  0.781 -1.763 -1.432 -0.930   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
19995  1.069  0.517 -0.690  0.241  0.913 -0.859  0.093 -0.359 -0.047  0.713   
19996 -0.529  0.438  0.672  1.436 -0.720  0.698 -0.350  2.150 -1.241 -0.167   
19997 -0.554 -0.936 -1.427  0.027 -0.539  0.994 -1.832 -1.156  0.474  1.483   
19998 -0.746  1.205  0.750 -0.236  1.139 -1.727 -0.677 -1.254 -0.099 -0.724   
19999  0.736 -0.216 -0.110 -1.404 -0.265 -1.770  0.7

c:\Python39\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
